# Effective Python Testing with Pytest

## What Makes `pytest` Useful?
With `pytest`, common tasks require less code and advanced tasks can be achieved through a variety of time-saving commands and plugins. It also runs existing tests out of the box.

This tutorial helps to understand some tools `pytest` provides to keep testing efficient and effective even as it scales.

### Less Boilerplate
Most functional tests follow the Arrange-Act-Assert model:
- **Arrange**: setup the conditions for the test
- **Act**: call some function or method
- **Assert**: assert that some end condition is true

Testing frameworks typically hook into your test's assertions so that they can provide information when an assertioin fails. Even a small set of tests requires a fair amount of boilerplate code.

```python
from unittest import TestCase

class TryTesting(TestCase):
    def test_always_passes(self):
        self.assertTrue(True)
    
    def test_always_fails(self):
        self.assertTrue(False)
```

These tests can be run from the command line using the `discover` option of `unittest`:

`python -m unittest discover`

A lot had to be done to run this test:
- import `TestCase` from `unittest`
- create `TryTesting` subclass
- write a method in `TryTesting` for each test
- use one of the `self.assert*` methods from `unittest.TestCase` to make assertions

This is a significant amount of code to write, and because it's the minimum you need for *any* test, you'd end up writing the same code over and over. `pytest` simplifies this workflow by allowing you to use Python's `assert` keyword directly:

In [7]:
# test_with_pytest.py

def test_always_passes():
    assert True
    
def test_always_fails():
    assert False

If you can write an expression that you expect to evaluate to `True`, then pytest will test it for you. It can be run using the `pytest` command:

In [9]:
!pytest

============================= test session starts ==============================
platform darwin -- Python 3.8.1, pytest-5.4.1, py-1.8.1, pluggy-0.13.1
rootdir: /Users/roytelles/Desktop/Python Codes/Real Python/Python Testing
collected 2 items                                                              

test_with_pytest.py .F                                                   [100%]

=================================== FAILURES ===================================
______________________________ test_always_fails _______________________________

    def test_always_fails():
>       assert False
E       assert False

test_with_pytest.py:5: AssertionError
=========================== short test summary info ============================
FAILED test_with_pytest.py::test_always_fails - assert False
========================= 1 failed, 1 passed in 0.13s ==========================


The `pytest` report shows:
- the system state, including which versions of Python, pytest, and any plugins you have installed
- the rootdir, the directory to search under for configuration and tests
- the number of test the runner discovered

The output then indicates the status of each test:
- **dot (.)** means the test passed
- **F** means the test failed
- **E** means the test raised an unexpected exception

For tests that fail, the report gives a detailed breakdown of the failure. In the example above, the test failed because `assert False` always fails. Finally, the report gives an overall status report of the test suite.

A few more assertion examples:

In [10]:
# test_with_pytest.py

def test_uppercase():
    assert "loud noises".upper() == "LOUD NOISES"
    
def test_reversed():
    assert list(reversed([1, 2, 3, 4])) == [4, 3, 2, 1]
    
def test_some_primes():
    assert 37 in {
        num
        
        for num in range(1, 50)
        if num != 1 and not any(
            [num % div == 0 for div in range(2, num)]
        )
    }

In [11]:
!pytest

============================= test session starts ==============================
platform darwin -- Python 3.8.1, pytest-5.4.1, py-1.8.1, pluggy-0.13.1
rootdir: /Users/roytelles/Desktop/Python Codes/Real Python/Python Testing
collected 5 items                                                              

test_with_pytest.py .F...                                                [100%]

=================================== FAILURES ===================================
______________________________ test_always_fails _______________________________

    def test_always_fails():
>       assert False
E       assert False

test_with_pytest.py:6: AssertionError
=========================== short test summary info ============================
FAILED test_with_pytest.py::test_always_fails - assert False
========================= 1 failed, 4 passed in 0.15s ==========================


The learning curve for `pytest` is shallower than for `unittest` because you don't need to learn new constructs for most tests. Also, the use of `assert` makes tests more understandable.

### State and Dependency Management
Tests should help you make your code more understandable. If the tests themselves are difficult to understand, then you may be in trouble.

`pytest` leads you toward **explicit** dependency declarations that are still reusable thanks to the availability of [fixtures](https://docs.pytest.org/en/latest/fixture.html). `pytest` fixtures are functions that create data or test doubles or initialize some system state for the test suite. Any test that wants to use a fixture must explicitly accept it as an argument, so dependencies are always stated up front.

Fixtures can also make use of other fixtures, again by declaring them explicitly as dependencies. Over time, your fixtures can become bulky and modular. Although the ability to insert fixtures into other fixtures provides enormous flexibility, it can also make managing dependencies more challenging as your test suite grows.

### Test Filtering
As your test suite grows, you may want to run just a few tests on a feature and save the full suite for later. `pytest` provides a few ways of doings this:
- **Name-based filtering**: You can limit pytest to running only those tests whose fully qualified names match a particular expression. You can do this with the `-k` parameter.
- **Directory scoping**: By default, `pytest` will run only those tests that are in or under the current directory
- **Test categorization**: `pytest` can include or exclude tests from particular categories that you define. You can do this with the `-m` parameter.

`pytest` enables you to create **marks**, or custom labels, for any test you like. A test may have multiple labels, and you can use them for granular control over which tests to run.

### Test Parameterization
When testing functions that process data or perform generic transofrmatoins, you'll be writing many similar tests. They may differ only in input or output of the code being tested. This requires duplicate test code, and doing so can sometimes obscure the behavior you're trying to test.

`pytest` offers a solution in which each test can pass or fail independently.

### Plugin-Based Architecture
`pytest` has a rich ecosystem of helpful plugins.

## Fixtures: Managing State and Dependencies
`pytest` fixtures are a way of providing data, test doubles, or state setup to your tests. Fixtures are functions that can return a wide range of values. Each test that depends on a fixture must explicitly accept that fixture as an argument.

### When to Create Fixtures
Imagine you're writing a function, `format_data_for_display()`, to process the data returned by an API endpoint. The data represents a list of people, each with a given name, family name, and job title. The function should output a list of strings that include each person's full name (their `given_name` followed by their `family_name`), a colon, and their `title`. To test this, you might write the following code:

In [1]:
# format_data.py

def format_data_for_display(people):
    '''
    Formats data returned from an API endpoint.
    
    Parameters
    ----------
    people : list
        A list of people with a given name, family name, and
        job title
        
    Returns
    -------
    list
        A list of strings formatted to include the person's full
        name (given name followed by family name) and their title.
    '''
    return [
        f"{person['given_name']} {person['family_name']}:"
        f" {person['title']}"
        
        for person in people
    ]


def test_format_data_for_display():
    '''
    Tests format_data_for_display function.
    '''
    people = [
        {
            "given_name": "Alfonsa",
            "family_name": "Ruiz",
            "title": "Senior Software Engineer",
        },
        {
            "given_name": "Sayid",
            "family_name": "Khan",
            "title": "Project Manager",
        },
    ]

    assert format_data_for_display(people) == [
        "Alfonsa Ruiz: Senior Software Engineer",
        "Sayid Khan: Project Manager",
    ]

In [3]:
!pytest -s format_data.py

============================= test session starts ==============================
platform darwin -- Python 3.8.1, pytest-5.4.1, py-1.8.1, pluggy-0.13.1
rootdir: /Users/roytelles/Desktop/Python Codes/Real Python/Python Testing
collected 1 item                                                               

format_data.py .

============================== 1 passed in 0.01s ===============================


Now suppose you need to write another function that transofrms the data into comma-separated values for use in Excel. The test looks familiar to the above:

In [14]:
# format_data_csv.py

def format_data_for_excel(people):
    '''
    Formats data returned from an API endpoint.
    
    Parameters
    ----------
    people : list
        A list of people with a given name, family name, and
        job title
        
    Returns
    -------
    str
        A string formatted to include the person's full
        name (given name followed by family name) and their title
        in comma-separated values.
    '''
    header = ','.join(people[0].keys())
    values = [','.join(person.values()) for person in people]
    
    # rejoin with new lines
    values = '\n'.join(values)
    
    return f"{header}\n{values}"

def test_format_data_for_excel():
    people = [
        {
            "given_name": "Alfonsa",
            "family_name": "Ruiz",
            "title": "Senior Software Engineer",
        },
        {
            "given_name": "Sayid",
            "family_name": "Khan",
            "title": "Project Manager",
        },
    ]

    assert format_data_for_excel(people) == (
        "given_name,family_name,title\n"
        "Alfonsa,Ruiz,Senior Software Engineer\n"
        "Sayid,Khan,Project Manager"
    )

In [15]:
!pytest -s format_data_csv.py

============================= test session starts ==============================
platform darwin -- Python 3.8.1, pytest-5.4.1, py-1.8.1, pluggy-0.13.1
rootdir: /Users/roytelles/Desktop/Python Codes/Real Python/Python Testing
collected 1 item                                                               

format_data_csv.py .

============================== 1 passed in 0.01s ===============================


If you find yourself writing several tests that all make use of underlying test data, then a fixture may be what you want. You can pull repeated data into a single function decorated with `@pytest.fixture` to indicate that the function is a `pytest` fixture:

In [16]:
import pytest

In [17]:
# data_format_fixture.py

@pytest.fixture
def example_people_data():
    return [
        {
            "given_name": "Alfonsa",
            "family_name": "Ruiz",
            "title": "Senior Software Engineer",
        },
        {
            "given_name": "Sayid",
            "family_name": "Khan",
            "title": "Project Manager",
        },
    ]

We can use this fixture by adding it as an argument to tests. Its value will be the return value of the fixture function:

In [19]:
# data_format_fixture.py

def test_format_data_for_display(example_people_data):
    assert format_data_for_display(example_people_data) == [
        "Alfonsa Ruiz: Senior Software Engineer",
        "Sayid Khan: Project Manager",
    ]

def test_format_data_for_excel(example_people_data):
    assert format_data_for_excel(example_people_data) == (
        "given_name,family_name,title\n"
        "Alfonsa,Ruiz,Senior Software Engineer\n"
        "Sayid,Khan,Project Manager"
    )

In [21]:
!pytest -s data_format_fixture.py

============================= test session starts ==============================
platform darwin -- Python 3.8.1, pytest-5.4.1, py-1.8.1, pluggy-0.13.1
rootdir: /Users/roytelles/Desktop/Python Codes/Real Python/Python Testing
collected 2 items                                                              

data_format_fixture.py ..

============================== 2 passed in 0.01s ===============================


Each test is now notably shorter but still has a clear path back to the data it depends on. Be sure to name fixtures something specific. That way it is easily determined if you want to use it when writing new tests in the future.